In [45]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import math
import matplotlib as mpl
from matplotlib.patches import Patch

# Compare DRT Result, Battery 42

## Load DRT Result

In [46]:
battery42_DRT_file = "../../EVC_EIS_Data/DRT_Result/data_CELL042_updated.csv" # check this path
battery42_DRT_df = pd.read_csv(battery42_DRT_file)

# Select columns of interest
cols_params = ['R1','R2','R3','R4',
               'ln_1_over_freq1','ln_1_over_freq2',
               'ln_1_over_freq3','ln_1_over_freq4']

# Sanity check: Charge_capacity is unique per date
df = battery42_DRT_df.rename(columns={'Charge_capacity_Ah':'charge_capacity'}).copy()
assert (df.groupby('date')['charge_capacity'].nunique() == 1).all(), "Assertion Error: Each date must have exactly one charge_capacity (SOH) value"


# Build SOH index (highest capacity = 1)
cap_map = (df[['date','charge_capacity']]
           .drop_duplicates()
           .sort_values('charge_capacity', ascending=False)
           .reset_index(drop=True))
cap_map['soh_idx'] = cap_map.index + 1
df = df.merge(cap_map, on=['date','charge_capacity'], how='left')

# Build SOC index (highest SOC = 1)
df['soc_key'] = df['soc']
df = df.sort_values(['soh_idx', 'soc_key'], ascending=[True, False]).copy()
df['soc_idx'] = (
    df.groupby('soh_idx')['soc_key']
      .transform(lambda s: pd.Categorical(s, categories=s.drop_duplicates().tolist(), ordered=True).codes + 1)
)

# ---------- final container ----------
keep_cols = ['date','charge_capacity','soc'] + cols_params
container = (df.set_index(['soh_idx','soc_idx'])[keep_cols].sort_index())

# Helpful lookups
soh_lookup = cap_map[['soh_idx','date','charge_capacity']].sort_values('soh_idx')
# per-SOH SOC lookup
soc_lookup = (df[['soh_idx','soc_idx','soc_key']]
              .drop_duplicates()
              .sort_values(['soh_idx','soc_idx']))



Note: Both SOH index and SOC index are 1-based index (start from 1). 

soh_idx = 1 means SOH is most healthy (Charge_capacity = highest);
soc_idx = 1 means SOC is the highest;

In [47]:
# Example Usage

# show first few pairs; soc_idx should start at 1 for each soh_idx
print(list(container.index.unique())[:15])

# what SOC indices exist for SOH=1?
print(soc_lookup.query('soh_idx == 1'))

soh1_soc1 = container.loc[(1, 1)]
print(soh1_soc1['R1']) # Highest SOH and Highest SOC's R1 value

print(len(soc_lookup.query('soh_idx == 5')['soc_idx']))

print(soh_lookup.query('soh_idx == 1')['charge_capacity'])


[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 1), (2, 2), (2, 3), (2, 4)]
    soh_idx  soc_idx   soc_key
0         1        1  0.913348
1         1        2  0.829059
2         1        3  0.744767
3         1        4  0.660466
4         1        5  0.576220
5         1        6  0.491863
6         1        7  0.407501
7         1        8  0.323126
8         1        9  0.238844
9         1       10  0.154577
10        1       11  0.070333
0.0278012593256643
13
0    3.57712
Name: charge_capacity, dtype: float64


## Visualization, CM7, R1, R2, R3, R4, freq1 ~ freq4

In [51]:
# ---- Configurations ----
ECM_MODEL = 'v3CM7'
OBJ_FUNC = 'RMSE_abs'
NUM_TRIALS = 50
REMOVE_OUTLIERS = True
remove_config = 'rmoutliers' #'c90filtered' # Change to '' when no remove

BASE_DIR = f"trial_results_{NUM_TRIALS}/objFunc_{OBJ_FUNC}"
FNAME_TEMPLATE = "battery42_soh{soh}_cycle{soc}_trials{num_trials}_objFunc_{obj}_{ecm}.csv"
if REMOVE_OUTLIERS:
    FNAME_TEMPLATE = "battery42_soh{soh}_cycle{soc}_trials{num_trials}_objFunc_{obj}_{ecm}_{remove_config}.csv"


# Which SOH blocks to plot (order = concat order on x-axis)
soh_to_plot = [1, 5]   # add more later, e.g. [1, 3, 5, 7]

# Parameters to plot
params_names = ['R1', 'R2', 'R3', 'R4', 'freq1', 'freq2', 'freq3', 'freq4']

# Order in CSV's `estimated_params`
if ECM_MODEL == 'v3CM7' or ECM_MODEL == 'v3CM9':
    CSV_PARAM_ORDER = ['R0','R1','R2','R3','C1','n1','C2','n2','C3','n3','Aw']
elif ECM_MODEL == 'v3CM8':
    CSV_PARAM_ORDER = ['R0','R1','R2','R3', 'R4', 'C1','n1','C2','n2','C3','n3','C4','n4','Aw']

In [52]:
# ---------- helpers ----------
def parse_estimated_params(row):
    """Turn the string in `estimated_params` into a dict of param->value."""
    try:
        vals = ast.literal_eval(row['estimated_params'])
        n = min(len(vals), len(CSV_PARAM_ORDER))
        return {CSV_PARAM_ORDER[i]: vals[i] for i in range(n)}
    except Exception:
        return {}

def compute_freq_from_params(k, pdict):
    """freq_k = 1 / (2*pi * (Rk * Ck)^(1/nk)); NaN if missing/invalid."""
    R = pdict.get(f'R{k}')
    C = pdict.get(f'C{k}')
    n = pdict.get(f'n{k}')
    try:
        if R is None or C is None or n is None: return np.nan
        if R <= 0 or C <= 0 or n <= 0: return np.nan
        return 1.0 / (2.0 * math.pi * (R * C) ** (1.0 / n))
    except Exception:
        return np.nan

def drt_value_for_param(container, soh_idx, soc_idx, pname):
    """
    DRT overlay value for (soh_idx, soc_idx, param).
    - Rk: read directly from container
    - freqk: 1 / exp(ln_1_over_freqk) from container
    """
    try:
        sub = container.loc[(soh_idx, soc_idx)]
        row = sub.iloc[0] if isinstance(sub, pd.DataFrame) else sub

        if pname.startswith('R'):
            return row.get(pname, np.nan)

        if pname.startswith('freq'):
            k = int(pname[-1])
            ln_name = f'ln_1_over_freq{k}'
            if ln_name in row and pd.notna(row[ln_name]):
                return 1.0 / math.exp(row[ln_name])
            return np.nan
    except KeyError:
        return np.nan
    except Exception:
        return np.nan

def load_trials_for_soh(soh_idx, soc_rows):
    """
    Read all cycle CSVs for a given SOH into long-form:
    Columns: [soh_idx, soc_idx, soc_value, param, value]
    """
    records = []
    for _, r in soc_rows.iterrows():
        soc_idx = int(r['soc_idx'])
        soc_value = r['soc_key']
        fname = os.path.join(BASE_DIR, FNAME_TEMPLATE.format(soh=soh_idx, soc=soc_idx, num_trials=NUM_TRIALS, obj=OBJ_FUNC, ecm=ECM_MODEL, remove_config=remove_config))
        if not os.path.exists(fname):
            # silently skip missing cycles
            print(f"Error: Can not find estimation_result_file: {fname}")
            continue
        est_df = pd.read_csv(fname)
        if 'estimated_params' not in est_df.columns:
            print(f"Error: Can not find 'estimated_params' column in estimation_result_file: {fname}")
            continue

        for _, tr in est_df.iterrows():
            pdict = parse_estimated_params(tr)

            # Add Rk values
            for p in [p for p in params_names if p.startswith('R')]:
                records.append({
                    'soh_idx': soh_idx, 'soc_idx': soc_idx, 'soc_value': soc_value,
                    'param': p, 'value': pdict.get(p, np.nan)
                })

            # Add freqs derived from trials
            for k in [1,2,3,4]:
                pname = f'freq{k}'
                if pname in params_names:
                    records.append({
                        'soh_idx': soh_idx, 'soc_idx': soc_idx, 'soc_value': soc_value,
                        'param': pname, 'value': compute_freq_from_params(k, pdict)
                    })
    return pd.DataFrame.from_records(records)

def build_drt_for_soh(container, soh_idx, soc_rows, params_names):
    """DRT overlays; columns: [soh_idx, soc_idx, soc_value, param, drt]."""
    out = []
    for _, r in soc_rows.iterrows():
        soc_idx = int(r['soc_idx'])
        soc_value = r['soc_key']
        for p in params_names:
            out.append({
                'soh_idx': soh_idx,
                'soc_idx': soc_idx,
                'soc_value': soc_value,
                'param': p,
                'drt': drt_value_for_param(container, soh_idx, soc_idx, p),
            })
    return pd.DataFrame(out)

def label_for(soh_idx, soc_val):
    # charge_capacity string (keep 5 decimals to match your example style)
    cap_lookup = soh_lookup.set_index('soh_idx')['charge_capacity'].to_dict()
    cap = cap_lookup.get(soh_idx, None)
    if cap is None or pd.isna(cap):
        cap_str = "NA"
    else:
        cap_str = f"{float(cap):.5f}"

    # SOC as percentage with 2 decimals. If your SOC is already 0–100, this will show e.g. 80.00%
    # If your SOC is 0–1 instead, change to: soc_float *= 100 before formatting.
    try:
        soc_float = float(soc_val)
        soc_float *= 100.0 # to percentage
    except Exception:
        soc_float = float("nan")
    soc_str = f"{soc_float:.2f}%"

    return f"SOH{cap_str}-SOC{soc_str}"

Gather data and plotting

In [53]:
# ---------- build data ----------

# sanity: soc_lookup should be per-SOH with local soc_idx
assert {'soh_idx','soc_idx','soc_key'}.issubset(soc_lookup.columns), \
    "soc_lookup must include ['soh_idx','soc_idx','soc_key'] built per SOH."

# Build a lookup: soh_idx -> charge_capacity (for x-axis labels & legend)
# expects soh_lookup has columns ['soh_idx','charge_capacity']
assert {'soh_idx','charge_capacity'}.issubset(soh_lookup.columns), \
    "soh_lookup must include ['soh_idx','charge_capacity']."
cap_lookup = soh_lookup.set_index('soh_idx')['charge_capacity'].to_dict()


# Collect trials & DRT overlays in the desired x-order: SOH blocks concatenated
trials_all, drt_all, cats = [], [], []  # cats: list of (soh_idx, soc_idx, soc_value)

for soh in soh_to_plot:
    soc_rows = (soc_lookup.loc[soc_lookup['soh_idx'] == soh]
                .sort_values('soc_idx'))  # soc_idx asc => high SOC -> low SOC
    trials_all.append(load_trials_for_soh(soh, soc_rows))
    drt_all.append(build_drt_for_soh(container, soh, soc_rows, params_names))
    cats.extend([(soh, int(r['soc_idx']), r['soc_key']) for _, r in soc_rows.iterrows()])

trials_long = (pd.concat(trials_all, ignore_index=True)
               if len(trials_all) else pd.DataFrame(columns=['soh_idx','soc_idx','soc_value','param','value']))
drt_long = (pd.concat(drt_all, ignore_index=True)
            if len(drt_all) else pd.DataFrame(columns=['soh_idx','soc_idx','soc_value','param','drt']))

# X tick labels in requested format
xtick_labels = [label_for(soh, soc_val) for (soh, _soc_idx, soc_val) in cats]


# ---------- plotting (SOH-colored boxplots + matching-color DRT dots) ----------
# choose a palette; cycles if >10 SOH blocks
palette = mpl.colormaps['tab10'].colors
soh_list_in_order = soh_to_plot[:]  # keep your chosen order
soh_color = {soh: palette[i % len(palette)] for i, soh in enumerate(soh_list_in_order)}

os.makedirs(f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials", exist_ok=True)
if REMOVE_OUTLIERS:
    os.makedirs(f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials/{remove_config}", exist_ok=True)
    

for pname in params_names:
    # prepare per-position data and track which SOH each position belongs to
    data_series = []
    pos_soh = []

    for (soh, soc_idx, soc_val) in cats:
        vals = trials_long.loc[
            (trials_long['soh_idx'] == soh) &
            (trials_long['soc_idx'] == soc_idx) &
            (trials_long['param'] == pname),
            'value'
        ].dropna().values.tolist()
        data_series.append(vals if len(vals) > 0 else [np.nan])
        pos_soh.append(soh)

    # group contiguous positions by SOH (cats was built block-wise per SOH)
    pos_by_soh = {}
    for pos, soh in enumerate(pos_soh, start=1):
        pos_by_soh.setdefault(soh, []).append(pos)

    plt.figure(figsize=(max(16, len(cats)*0.6), 5))

    # draw one boxplot call per SOH using its color
    for soh, positions in pos_by_soh.items():
        vals = [data_series[p-1] for p in positions]
        plt.boxplot(
            vals,
            positions=positions,
            showfliers=False,
            patch_artist=True,
            boxprops=dict(facecolor=soh_color[soh], alpha=0.6),
            medianprops=dict(color='black'),
        )

    # overlay DRT points at each position with matching SOH color
    for pos, (soh, soc_idx, _soc_val) in enumerate(cats, start=1):
        d = drt_long.loc[
            (drt_long['param'] == pname) &
            (drt_long['soh_idx'] == soh) &
            (drt_long['soc_idx'] == soc_idx),
            'drt'
        ]
        if not d.empty and pd.notna(d.iloc[0]):
            plt.scatter([pos], [float(d.iloc[0])], zorder=3, color=soh_color[soh])

    # axes, labels, legend
    plt.xticks(range(1, len(cats)+1), xtick_labels, rotation=45, ha='right')
    plt.xlabel("SOC (high → low), grouped by SOH")
    plt.ylabel(pname)
    plt.title(f"battery42; {pname} across SOC for different SOH with DRT Result overlays \n {ECM_MODEL} with objFunc={OBJ_FUNC} (#trials={NUM_TRIALS} {remove_config})")

    legend_handles = []
    for soh in soh_list_in_order:
        cap = cap_lookup.get(soh, None)
        label = f"SOH{soh}" if cap is None or pd.isna(cap) else f"SOH{soh} ({float(cap):.5f})"
        legend_handles.append(Patch(facecolor=soh_color[soh], edgecolor='black', label=label))
    plt.legend(handles=legend_handles, title="SOH blocks", loc='best')

    plt.tight_layout()
    folder_path = f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials"
    file_path = f"battery42_{pname}_DRTvsOptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_boxplots.png"
    if REMOVE_OUTLIERS:
        folder_path = f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials/{remove_config}"
        file_path = f"battery42_{pname}_DRTvsOptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_{remove_config}_boxplots.png"
    out_path = os.path.join(folder_path, file_path)
    plt.savefig(out_path, dpi=200)
    plt.close()

print(f"Done. Figures saved under DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials.")

## Plotting without DRT
for pname in params_names:
    data_series = []
    pos_soh = []

    for (soh, soc_idx, soc_val) in cats:
        vals = trials_long.loc[
            (trials_long['soh_idx'] == soh) &
            (trials_long['soc_idx'] == soc_idx) &
            (trials_long['param'] == pname),
            'value'
        ].dropna().values.tolist()
        data_series.append(vals if len(vals) > 0 else [np.nan])
        pos_soh.append(soh)

    # group contiguous positions by SOH
    pos_by_soh = {}
    for pos, soh in enumerate(pos_soh, start=1):
        pos_by_soh.setdefault(soh, []).append(pos)

    plt.figure(figsize=(max(16, len(cats)*0.6), 5))

    # draw one boxplot call per SOH using its color
    for soh, positions in pos_by_soh.items():
        vals = [data_series[p-1] for p in positions]
        plt.boxplot(
            vals,
            positions=positions,
            showfliers=False,
            patch_artist=True,
            boxprops=dict(facecolor=soh_color[soh], alpha=0.6),
            medianprops=dict(color='black'),
        )

    # x labels and legend
    plt.xticks(range(1, len(cats)+1), xtick_labels, rotation=45, ha='right')
    plt.xlabel("SOC (high → low), grouped by SOH")
    plt.ylabel(pname)
    plt.title(f"battery42; {pname} across SOC for different SOH \n {ECM_MODEL} with objFunc={OBJ_FUNC} (#trials={NUM_TRIALS} {remove_config})")

    legend_handles = []
    for soh in soh_list_in_order:
        cap = cap_lookup.get(soh, None)
        label = f"SOH{soh}" if cap is None or pd.isna(cap) else f"SOH{soh} ({float(cap):.5f})"
        legend_handles.append(Patch(facecolor=soh_color[soh], edgecolor='black', label=label))
    plt.legend(handles=legend_handles, title="SOH blocks", loc='best')

    plt.tight_layout()
    folder_path = f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials"
    file_path = f"battery42_{pname}_OptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_boxplots.png"
    if REMOVE_OUTLIERS:
        folder_path = f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials/{remove_config}"
        file_path = f"battery42_{pname}_OptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_{remove_config}_boxplots.png"
    out_path = os.path.join(folder_path, file_path)
    plt.savefig(out_path, dpi=200)
    plt.close()

print(f"Done. Boxplot-only figures saved under DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials with *_OptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_boxplots.png.")

Done. Figures saved under DRT_Comparison/v3CM7/RMSE_abs_50trials.
Done. Boxplot-only figures saved under DRT_Comparison/v3CM7/RMSE_abs_50trials with *_OptFitting_v3CM7_RMSE_abs_50trials_boxplots.png.


Stats: Comparison between DRT result and Algorithm Fitting Result

In [54]:
# ---------- stats (boxplots + DRT) ----------

from itertools import product

# Keys we’ll use everywhere
KEYS3 = ['soh_idx','soc_idx','param']

# Map (soh_idx, soc_idx) -> x-position and pretty x-label
pos_map = {}
label_map = {}
for i, (soh, soc_idx, soc_val) in enumerate(cats, start=1):
    pos_map[(soh, soc_idx)] = i
    label_map[(soh, soc_idx)] = label_for(soh, soc_val)

# Base grid: all (position, param) we showed on the x-axis,
# so even missing cycles/CSVs appear in the table (with NaNs)
base_rows = []
for (soh, soc_idx, soc_val) in cats:
    for pname in params_names:
        base_rows.append({
            'soh_idx': soh,
            'soc_idx': soc_idx,
            'soc_value': soc_val,
            'param': pname,
            'x_pos': pos_map[(soh, soc_idx)],
            'x_label': label_map[(soh, soc_idx)],
        })
base = pd.DataFrame(base_rows)

# ---- Trial boxplot stats (count, min, Q1, median, Q3, max, mean, std) ----
if not trials_long.empty:
    g = trials_long.groupby(['soh_idx','soc_idx','param'], as_index=False)
    trial_basic = g['value'].agg(n='count', mean='mean', std='std', min='min', max='max', median='median')

    # quantiles
    q1 = g['value'].quantile(0.25).rename(columns={'value':'q1'})
    q3 = g['value'].quantile(0.75).rename(columns={'value':'q3'})
    # merge them
    trial_stats = trial_basic.merge(q1, on=KEYS3, how='left').merge(q3, on=KEYS3, how='left')
    trial_stats['iqr'] = trial_stats['q3'] - trial_stats['q1']
    trial_stats['coef_of_var'] = trial_stats['std']/trial_stats['mean']
else:
    trial_stats = pd.DataFrame(columns=KEYS3 + ['n','mean','std','min','max','median','q1','q3','iqr', 'coef_of_var'])

# ---- DRT values (one per SOH×SOC×param) ----
if not drt_long.empty:
    drt_stats = (drt_long
                 .groupby(KEYS3, as_index=False)['drt']
                 .median()  # median == single value; robust if dupes
                 .rename(columns={'drt':'drt_value'}))
else:
    drt_stats = pd.DataFrame(columns=KEYS3 + ['drt_value'])

# ---- Combine on our full x-grid ----
stats_all = (base
             .merge(trial_stats, on=KEYS3, how='left')
             .merge(drt_stats, on=KEYS3, how='left'))

# DRT vs trials comparison metrics
stats_all['drt_minus_median'] = stats_all['drt_value'] - stats_all['median']
# stats_all['drt_rel_to_median'] = stats_all['drt_value'] / stats_all['median'] - 1.0
# stats_all.loc[stats_all['median'].isna() | (stats_all['median'] == 0), 'drt_rel_to_median'] = np.nan
# stats_all['drt_zscore_vs_trials'] = (stats_all['drt_value'] - stats_all['mean']) / stats_all['std']
# stats_all.loc[stats_all['std'].isna() | (stats_all['std'] == 0), 'drt_zscore_vs_trials'] = np.nan

# Order rows by parameter then x-position (same visual order as plots)
stats_all = stats_all.sort_values(['param', 'x_pos']).reset_index(drop=True)

# Nice column order
# cols_order = ['param','x_pos','x_label','soh_idx','soc_idx','soc_value',
#               'n','min','q1','median','q3','max','mean','std','iqr',
#               'drt_value','drt_minus_median','drt_rel_to_median','drt_zscore_vs_trials']

cols_order = ['param','x_pos','x_label','soh_idx','soc_idx','soc_value',
              'n','min','q1','median','q3','max','mean','std','iqr','coef_of_var',
              'drt_value','drt_minus_median']
stats_all = stats_all[cols_order]

# Round for readability (without losing raw precision in CSVs)
def _rnd(x): 
    return x if pd.isna(x) else float(f"{x:.6g}")
for c in ['min','q1','median','q3','max','mean','std','iqr','coef_of_var','drt_value','drt_minus_median']:
    if c in stats_all.columns:
        stats_all[c] = stats_all[c].map(_rnd)

# ---- Save CSVs ----
folder_path = f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials"
file_path = f"battery42_DRTvsOptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_AllParams_stats.csv"
if REMOVE_OUTLIERS:
    folder_path = f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials/{remove_config}"
    file_path = f"battery42_DRTvsOptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_{remove_config}_AllParams_stats.csv"

stats_all.to_csv(os.path.join(folder_path, file_path), index=False)

# # also per-parameter CSVs
# for p in params_names:
#     sub = stats_all[stats_all['param'] == p]
#     sub.to_csv(os.path.join(f"DRT_Comparison/{ECM_MODEL}/{OBJ_FUNC}_{NUM_TRIALS}trials", f"battery42_DRTvsOptFitting_{ECM_MODEL}_{OBJ_FUNC}_{NUM_TRIALS}trials_{p}_stats.csv"), index=False)


# ---- Console print (grouped by param), compact view ----
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

for p in params_names:
    sub = stats_all[stats_all['param'] == p].copy()
    # Keep the most useful columns for a quick read
    view_cols = ['x_pos','x_label','n','min','q1','median','q3','max','mean','std',
                 'drt_value','drt_minus_median']
    print("\n" + "="*120)
    print(f"Parameter: {p}")
    print(sub[view_cols].to_string(index=False))
    print("="*120)

print("Stats saved to *all_params_stats.csv and per-parameter CSVs.")



Parameter: R1
 x_pos              x_label  n      min       q1   median       q3      max     mean      std  drt_value  drt_minus_median
     1 SOH3.57712-SOC91.33% 18 0.008460 0.009508 0.011785 0.013931 0.015689 0.011863 0.002424   0.027801          0.016016
     2 SOH3.57712-SOC82.91% 19 0.004840 0.009505 0.011783 0.013556 0.019458 0.012375 0.003955   0.027730          0.015947
     3 SOH3.57712-SOC74.48% 14 0.007867 0.010959 0.013755 0.015607 0.018600 0.013288 0.003297   0.027652          0.013897
     4 SOH3.57712-SOC66.05% 20 0.007011 0.009977 0.013636 0.015692 0.020057 0.013131 0.003701   0.027909          0.014274
     5 SOH3.57712-SOC57.62% 12 0.008181 0.009764 0.011226 0.016522 0.016915 0.012549 0.003386   0.027987          0.016761
     6 SOH3.57712-SOC49.19% 23 0.006691 0.012671 0.013901 0.016610 0.021209 0.014284 0.003433   0.027447          0.013546
     7 SOH3.57712-SOC40.75% 23 0.004264 0.011456 0.013594 0.016931 0.022882 0.014064 0.003972   0.027932          0.014338
 

In [54]:
# # ---------- plotting (SOH-colored boxplots + matching-color DRT dots + numeric annotations) ----------
# import matplotlib as mpl
# from matplotlib.patches import Patch

# # ---- annotation config ----
# ANNOTATE_STATS = True
# STATS_FONT_SIZE = 8
# STATS_SIGFIGS = 4   # how many significant figures to display
# Q1_Y_OFFSET_PTS = -10  # label offset for Q1 (in points)
# MED_Y_OFFSET_PTS =  4  # label offset for median
# Q3_Y_OFFSET_PTS =  10  # label offset for Q3
# DRT_Y_OFFSET_PTS =  6  # label offset for DRT dot

# def _fmt_num(x):
#     try:
#         if x is None or (isinstance(x, float) and (np.isnan(x) or np.isinf(x))):
#             return "NA"
#         # Significant figures formatting (handles wide ranges better than fixed decimals)
#         return f"{float(x):.{STATS_SIGFIGS}g}"
#     except Exception:
#         return "NA"

# # Precompute trial stats for all parameters (so we can annotate)
# if not trials_long.empty:
#     g = trials_long.groupby(['param','soh_idx','soc_idx'])
#     trial_n      = g['value'].count().rename('n')
#     trial_median = g['value'].median().rename('median')
#     trial_q1     = g['value'].quantile(0.25).rename('q1')
#     trial_q3     = g['value'].quantile(0.75).rename('q3')
#     trial_stats_df = pd.concat([trial_n, trial_median, trial_q1, trial_q3], axis=1).reset_index()

#     # nested dict: stats_by_param[pname][(soh, soc_idx)] -> {'n':..., 'median':..., 'q1':..., 'q3':...}
#     stats_by_param = {
#         p: sub.set_index(['soh_idx','soc_idx'])[['n','median','q1','q3']].to_dict('index')
#         for p, sub in trial_stats_df.groupby('param')
#     }
# else:
#     stats_by_param = {}

# # choose a palette; cycles if >10 SOH blocks
# palette = mpl.colormaps['tab10'].colors
# soh_list_in_order = soh_to_plot[:]  # keep your chosen order
# soh_color = {soh: palette[i % len(palette)] for i, soh in enumerate(soh_list_in_order)}

# os.makedirs(f"DRT_Comparison2/{ECM_MODEL}", exist_ok=True)

# for pname in params_names:
#     # prepare per-position data and track which SOH each position belongs to
#     data_series = []
#     pos_soh = []

#     for (soh, soc_idx, soc_val) in cats:
#         vals = trials_long.loc[
#             (trials_long['soh_idx'] == soh) &
#             (trials_long['soc_idx'] == soc_idx) &
#             (trials_long['param'] == pname),
#             'value'
#         ].dropna().values.tolist()
#         data_series.append(vals if len(vals) > 0 else [np.nan])
#         pos_soh.append(soh)

#     # group contiguous positions by SOH (cats was built block-wise per SOH)
#     pos_by_soh = {}
#     for pos, soh in enumerate(pos_soh, start=1):
#         pos_by_soh.setdefault(soh, []).append(pos)

#     plt.figure(figsize=(max(16, len(cats)*0.6), 5))

#     # draw one boxplot call per SOH using its color
#     for soh, positions in pos_by_soh.items():
#         vals = [data_series[p-1] for p in positions]
#         plt.boxplot(
#             vals,
#             positions=positions,
#             showfliers=False,
#             patch_artist=True,
#             boxprops=dict(facecolor=soh_color[soh], alpha=0.6),
#             medianprops=dict(color='black'),
#         )

#     # overlay DRT points at each position with matching SOH color
#     for pos, (soh, soc_idx, _soc_val) in enumerate(cats, start=1):
#         d = drt_long.loc[
#             (drt_long['param'] == pname) &
#             (drt_long['soh_idx'] == soh) &
#             (drt_long['soc_idx'] == soc_idx),
#             'drt'
#         ]
#         if not d.empty and pd.notna(d.iloc[0]):
#             y = float(d.iloc[0])
#             plt.scatter([pos], [y], zorder=3, color=soh_color[soh])
#             if ANNOTATE_STATS:
#                 plt.annotate(
#                     f"DRT={_fmt_num(y)}",
#                     xy=(pos, y),
#                     xycoords='data',
#                     textcoords='offset points',
#                     xytext=(0, DRT_Y_OFFSET_PTS),
#                     ha='center', va='bottom',
#                     fontsize=STATS_FONT_SIZE,
#                     color=soh_color[soh]
#                 )

#     # annotate box stats (median, q1, q3) at each position
#     if ANNOTATE_STATS:
#         for pos, (soh, soc_idx, _soc_val) in enumerate(cats, start=1):
#             stats_map = stats_by_param.get(pname, {})
#             st = stats_map.get((soh, soc_idx))
#             if not st:
#                 continue
#             med = st.get('median', np.nan)
#             q1  = st.get('q1', np.nan)
#             q3  = st.get('q3', np.nan)

#             if pd.notna(q1):
#                 plt.annotate(
#                     f"q1={_fmt_num(q1)}",
#                     xy=(pos, q1),
#                     xycoords='data',
#                     textcoords='offset points',
#                     xytext=(0, Q1_Y_OFFSET_PTS),
#                     ha='center', va='top',
#                     fontsize=STATS_FONT_SIZE
#                 )
#             if pd.notna(med):
#                 plt.annotate(
#                     f"med={_fmt_num(med)}",
#                     xy=(pos, med),
#                     xycoords='data',
#                     textcoords='offset points',
#                     xytext=(0, MED_Y_OFFSET_PTS),
#                     ha='center', va='bottom',
#                     fontsize=STATS_FONT_SIZE
#                 )
#             if pd.notna(q3):
#                 plt.annotate(
#                     f"q3={_fmt_num(q3)}",
#                     xy=(pos, q3),
#                     xycoords='data',
#                     textcoords='offset points',
#                     xytext=(0, Q3_Y_OFFSET_PTS),
#                     ha='center', va='bottom',
#                     fontsize=STATS_FONT_SIZE
#                 )

#     # axes, labels, legend
#     plt.xticks(range(1, len(cats)+1), xtick_labels, rotation=45, ha='right')
#     plt.xlabel("SOC (high → low), grouped by SOH")
#     plt.ylabel(pname)
#     plt.title(f"battery42; {pname} across SOC for different SOH with DRT Result overlays \n {ECM_MODEL} with objFunc={OBJ_FUNC}")

#     legend_handles = []
#     for soh in soh_list_in_order:
#         cap = cap_lookup.get(soh, None)
#         label = f"SOH{soh}" if cap is None or pd.isna(cap) else f"SOH{soh} ({float(cap):.5f})"
#         legend_handles.append(Patch(facecolor=soh_color[soh], edgecolor='black', label=label))
#     plt.legend(handles=legend_handles, title="SOH blocks", loc='best')

#     plt.tight_layout()
#     out_path = os.path.join(f"DRT_Comparison2/{ECM_MODEL}", f"battery42_{pname}_DRTvsOptFitting_{ECM_MODEL}_boxplots.png")
#     plt.savefig(out_path, dpi=200)
#     plt.close()

# print(f"Done. Figures saved under DRT_Comparison/{ECM_MODEL}/ with stats annotations.")


Done. Figures saved under DRT_Comparison/V3CM7/ with stats annotations.
